# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [7]:
import pandas as pd
import numpy as np
import multiprocessing

## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `policy_priority_inference.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import PPI. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `policy_priority_inference.py` file into the folder where this tutorial is located.

In [2]:
import requests # the Python library that helps placing requests to websites
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/policy_priority_inference.py'
r = requests.get(url)
with open('policy_priority_inference.py', 'w') as f:
    f.write(r.text)
import policy_priority_inference as ppi

## Load data

### Indicators

In [3]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')

N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix

### Interdependency network

In [4]:
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')

A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

### Budget

In [5]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')

Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

### Budget-indicator mapping

In [6]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')

B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [13]:
T = Bs.shape[1]
parallel_processes = multiprocessing.cpu_count() # number of cores to use
threshold = 0.6 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 50 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl,Bs=Bs, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -356999.9999992858
Iteration: 2 .    Worst goodness of fit: -107999.9999997839
Iteration: 3 .    Worst goodness of fit: -18162.870689667605
Iteration: 4 .    Worst goodness of fit: -162068.31249967575
Iteration: 5 .    Worst goodness of fit: -12513.624999974958
Iteration: 6 .    Worst goodness of fit: -25154.296874949672
Iteration: 7 .    Worst goodness of fit: -3107.5096982779896
Iteration: 8 .    Worst goodness of fit: -9294.433593731403
Iteration: 9 .    Worst goodness of fit: -2672.621582025898
Iteration: 10 .    Worst goodness of fit: -3070.1293945251077
Iteration: 11 .    Worst goodness of fit: -726.5995483383858
Iteration: 12 .    Worst goodness of fit: -1170.7649230933607
Iteration: 13 .    Worst goodness of fit: -477.01248931788916
Iteration: 14 .    Worst goodness of fit: -497.43604659934664
Iteration: 15 .    Worst goodness of fit: -90.81301498394517
Iteration: 16 .    Worst goodness of fit: -167.37627983059772
Iteration: 17 .    Wors

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [14]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [15]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.001133245803225464,0.0007173762481386406,0.24241409890285115,69,7.590668121271182e-05,0.0007531993342749926,0.9979937255954184,0.9917148073229751
1,0.020458032143704483,0.007079263090817033,0.0848783259666791,nan,-0.0008694787331604004,0.00036435186515682005,0.9977300001428565,0.9927129626968636
2,5.1693076043971075e-05,1.3811444084734423e-07,0.09593643744592241,nan,7.540400681549375e-05,0.0011786988454745018,0.765987565055204,0.9870343126997805
3,0.0005761690086828743,5.948271547088215e-06,0.45278488598703287,nan,0.0016094993539530478,0.007174241233964651,0.8659981042465701,0.9774523846932539
4,0.0031935781830767103,5.352177775543664e-05,0.630040148291449,nan,0.0008916166336202513,0.0029821886752324223,0.9906708133621611,0.9934391849144887
...,...,...,...,...,...,...,...,...
67,9.713561676005822e-05,1.9385719657236186e-05,1.2396820634166306,nan,6.840147565778665e-05,-0.003719733677633008,0.9865879459494535,0.9954536588384485
68,0.0017893447953184556,2.030241751547041e-05,0.30940965430059897,nan,0.002248920962028389,0.001570757429969094,0.9140118455695029,0.9930886673081359
69,0.0012896708569835869,0.0002254663424906005,1.127350357474036,nan,-0.003079506636798124,0.003278035523098688,0.9483304255570785,0.9954927011557393
70,0.007772478668108582,0.0005000410330631608,0.9027422007339055,nan,-0.005955435783710938,-0.003325886245951093,0.980084374817598,0.9943715771222366


## Save parameters data

In [ ]:
df_params.to_csv('clean_data/parameters.csv', index=False)